# 0. Install and Import dependencies

In [3]:
# Gym stuff
import gymnasium as gym
import gym_anytrading

# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'stable_baselines3'

In [ ]:
df = pd.read_csv('./UPS_1Y_Stock.csv')

In [ ]:
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

In [ ]:
df.set_index('Date', inplace=True)
df.head()

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(5,len(df)), window_size=5)

In [ ]:
env.signal_features

In [ ]:
env.action_space

In [ ]:
state = env.reset()
for _ in range(len(df)-6): # i have no clue why -6
    action = env.action_space.sample()
    #https://stackoverflow.com/questions/73195438/openai-gyms-env-step-what-are-the-values
    obs, reward, terminated, truncated , info = env.step(action)
    if terminated:
        print("terminated")
        break
        
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()

In [ ]:
env_maker = lambda: gym.make('stocks-v0', df=df, frame_bound=(5,100), window_size=5)
env = DummyVecEnv([env_maker])

In [ ]:
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)

# 4. Evaluation

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(90,110), window_size=5)
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()